# Product Mix

In [1]:
from pulp import *

x = LpVariable('x',lowBound = 0, cat='Continuous')
y = LpVariable('y', upBound = 5, cat='Integer')

f = LpAffineExpression(LpElement('x'))
print(f)

x_name = ['x_0', 'x_1', 'x_2']
x = [LpVariable(x_name[i], lowBound = 0, upBound = 10) for i in range(3)]
c = LpAffineExpression([(x[0],1), (x[1],-3), (x[2],4)])
print(c)

x
x_0 - 3*x_1 + 4*x_2


In [2]:
prob = LpProblem(name='Maximize', sense=LpMaximize)

In [3]:
x1 = LpVariable(name='X1', lowBound=0)
x2 = LpVariable(name='X2', lowBound=0)

$$\max Z= 3X_1+2X_2$$

In [4]:
prob += 3*x1 + 2*x2

$$\begin{split}
10X_1+5X_2&\le300\\
4X_1+4X_2&\le160\\
2X_1+6X_2&\le180\\
X_1,X_2&\ge0\end{split}$$

In [5]:
prob += 10*x1 + 5*x2 <= 300
prob += 4*x1 + 4*x2 <= 160
prob += 2*x1 + 6*x2 <= 180

In [6]:
prob.solve(GUROBI())

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 3e+02]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+30   4.875000e+30   5.000000e+00      0s
       2    1.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds
Optimal objective  1.000000000e+02
Gurobi status= 2


1

In [7]:
for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Status', LpStatus[prob.status])
print(pulp.value(prob.objective))

X1 = 20.0
X2 = 20.0
Status Optimal
100.0


### Problem 2

In [8]:
from pulp import *

prob = LpProblem(name='Mixed Products', sense=LpMinimize)

identifiers = ['X1', 'X2']
obj_coef = [200, 300]
protein = [1, 3]
starch = [1, 1]
fat = [4, 1]

prices = dict(zip(identifiers, obj_coef))
const1 = dict(zip(identifiers, protein))
const2 = dict(zip(identifiers, starch))
const3 = dict(zip(identifiers, fat))

var = LpVariable.dicts(name='X', indexs=identifiers, lowBound=0)

$$\min Z=200X_1+300X_2$$

In [9]:
prob += lpSum([var[i]*prices[i] for i in identifiers])

prob += lpSum([var[i]*const1[i] for i in identifiers]) >= 30
prob += lpSum([var[i]*const2[i] for i in identifiers]) >= 20
prob += lpSum([var[i]*const3[i] for i in identifiers]) >= 35

$$\begin{split}
X_1+3X_2&\ge30\\
X_1+X_2&\ge20\\
4X_1+X_2&\ge35\\
X_1,X_2&\ge0
\end{split}$$

In [10]:
prob.solve(GUROBI())

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Presolve time: 0.00s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.625000e+01   0.000000e+00      0s
       2    4.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  4.500000000e+03
Gurobi status= 2


1

In [11]:
for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Status:', LpStatus[prob.status])
print(pulp.value(prob.objective))

X_X1 = 15.0
X_X2 = 5.0
Status: Optimal
4500.0


In [12]:
from pulp import *

plants = ['P1', 'P2', 'P3']
warehouses = ['W1', 'W2', 'W3', 'W4']
costs = [[3, 2, 4, 1], 
         [2, 4, 3, 2], 
         [3, 1, 5, 3]]

supply = dict(zip(plants, [250, 180, 170]))
demand = dict(zip(warehouses, [130, 210, 160, 100]))
sd_costs = makeDict([plants, warehouses], costs, 0)

routes = [(p, w) for p in plants for w in warehouses]

prob = LpProblem(name='Transportation', sense=LpMinimize)

x = LpVariable.dicts(name='Route', indexs=(plants, warehouses), lowBound=0) 

prob += lpSum([x[p][w] * sd_costs[p][w] for (p, w) in routes])


for p in plants:
    prob += lpSum([x[p][w] for w in warehouses]) == supply[p]
    
for w in warehouses:
    prob += lpSum([x[p][w] for p in plants]) == demand[w]
    


In [13]:
prob.solve(solver=GUROBI())

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+02]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0500000e+03   3.000000e+02   0.000000e+00      0s
       2    1.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  1.200000000e+03
Gurobi status= 2


1

In [14]:
print('Status:', LpStatus[prob.status])
for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Optimal value:', value(prob.objective))

Status: Optimal
Route_P1_W1 = 110.0
Route_P1_W2 = 40.0
Route_P1_W3 = 0.0
Route_P1_W4 = 100.0
Route_P2_W1 = 20.0
Route_P2_W2 = 0.0
Route_P2_W3 = 160.0
Route_P2_W4 = 0.0
Route_P3_W1 = 0.0
Route_P3_W2 = 170.0
Route_P3_W3 = 0.0
Route_P3_W4 = 0.0
Optimal value: 1200.0


### Problem 4

In [15]:
from pulp import *

identifiers = ['A', 'B']
obj_coef = dict(zip(identifiers, [3, 2]))
const1 = dict(zip(identifiers, [10, 5]))
const2 = dict(zip(identifiers, [4, 4]))
const3 = dict(zip(identifiers, [2, 6]))
const4 = dict(zip(identifiers, [1, 0]))
const5 = dict(zip(identifiers, [1, -2]))

prob = LpProblem(name='Minimize', sense=LpMaximize)

x = LpVariable.dicts(name='Decision variables', indexs=identifiers, 
                     lowBound=0)

prob += lpSum(x[i] * obj_coef[i] for i in identifiers)

prob += lpSum(x[i] * const1[i] for i in identifiers) <= 300
prob += lpSum(x[i] * const2[i] for i in identifiers) <= 160
prob += lpSum(x[i] * const3[i] for i in identifiers) <= 180
prob += lpSum(x[i] * const4[i] for i in identifiers) >= 20
prob += lpSum(x[i] * const5[i] for i in identifiers) == 0

prob.solve(solver=GUROBI())

print('Status:', prob.status)
for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Optimal value:', value(prob.objective))

Optimize a model with 5 rows, 2 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.600000000e+01
Gurobi status= 2
Status: 1
Decision_variables_A = 24.0
Decision_variables_B = 12.0
Optimal value: 96.0


### Problem 5

In [16]:
from pulp import *

In [17]:
model = LpProblem(name='Product Planning', sense=LpMinimize)

In [18]:
var = 'X11 X12 X13 X21 X22 X23 Y11 Y12 Y13 Y21 Y22 Y23 I1 I2 I3 D1 D2 D3'.split()
coe = [30, 35, 30, 20, 25, 20, 0.3, 0.3, 0.3, 0.15, 0.15, 0.15, 0.5, 0.5, 0.5, 0.2, 0.2, 0.2]
obj = dict(zip(var, coe))
zero = [0 for i in range(len(var))]

In [19]:
x = LpVariable.dicts(name='X', indexs=var, indexStart=[], lowBound=0, cat='Continuous')

In [20]:
model += lpSum(x[i]*obj[i] for i in var)

In [21]:
const1 = dict(zip(var, zero))
const1['X11'] = 1
const1['Y11'] = -1

const2 = dict(zip(var, zero))
const2['X21'] = 1
const2['Y21'] = -1

const3 = dict(zip(var, zero))
const3['X12'] = 1
const3['Y11'] = 1
const3['Y12'] = -1

const4 = dict(zip(var, zero))
const4['X22'] = 1
const4['Y21'] = 1
const4['Y22'] = -1

const5 = dict(zip(var, zero))
const5['X13'] = 1
const5['Y12'] = 1
const5['Y13'] = -1

const6 = dict(zip(var, zero))
const6['X23'] = 1
const6['Y22'] = 1
const6['Y23'] = -1

const7 = dict(zip(var, zero))
const7['Y13'] = 1

const8 = dict(zip(var, zero))
const8['Y23'] = 1

const9 = dict(zip(var, zero))
const9['Y11'] = 1
const9['Y21'] = 1

const10 = dict(zip(var, zero))
const10['Y12'] = 1
const10['Y22'] = 1

const11 = dict(zip(var, zero))
const11['Y13'] = 1
const11['Y23'] = 1

const12 = dict(zip(var, zero))
const12['X11'] = 0.2
const12['X21'] = 0.1

const13 = dict(zip(var, zero))
const13['X12'] = 0.2
const13['X22'] = 0.1

const14 = dict(zip(var, zero))
const14['X13'] = 0.2
const14['X23'] = 0.1

const15 = dict(zip(var, zero))
const15['X11'] = 0.05
const15['X21'] = 0.04

const16 = dict(zip(var, zero))
const16['X12'] = 0.05
const16['X22'] = 0

const17 = dict(zip(var, zero))
const17['X13'] = 0.05
const17['X23'] = 0.04

const18 = dict(zip(var, zero))
const18['X11'] = 1
const18['X21'] = 1
const18['I1'] = -1
const18['D1'] = 1

const19 = dict(zip(var, zero))
const19['X11'] = -1
const19['X12'] = 1
const19['X21'] = -1
const19['X22'] = 1
const19['I2'] = -1
const19['D2'] = 1

const20 = dict(zip(var, zero))
const20['X12'] = -1
const20['X13'] = 1
const20['X22'] = -1
const20['X23'] = 1
const20['I3'] = -1
const20['D3'] = 1

In [22]:
model += lpSum([const1[i]*x[i] for i in var]) == 2500
model += lpSum([const2[i]*x[i] for i in var]) == 2000
model += lpSum([const3[i]*x[i] for i in var]) == 2500
model += lpSum([const4[i]*x[i] for i in var]) == 2000
model += lpSum([const5[i]*x[i] for i in var]) == 3000
model += lpSum([const6[i]*x[i] for i in var]) == 2000
model += lpSum([const7[i]*x[i] for i in var]) >= 500
model += lpSum([const8[i]*x[i] for i in var]) >= 500
model += lpSum([const9[i]*x[i] for i in var]) <= 2000
model += lpSum([const10[i]*x[i] for i in var]) <= 2000
model += lpSum([const11[i]*x[i] for i in var]) <= 2000
model += lpSum([const12[i]*x[i] for i in var]) <= 1000
model += lpSum([const13[i]*x[i] for i in var]) <= 1000
model += lpSum([const14[i]*x[i] for i in var]) <= 1000
model += lpSum([const15[i]*x[i] for i in var]) <= 300
model += lpSum([const16[i]*x[i] for i in var]) <= 300
model += lpSum([const17[i]*x[i] for i in var]) <= 300
model += lpSum([const18[i]*x[i] for i in var]) == 5000
model += lpSum([const19[i]*x[i] for i in var]) == 0
model += lpSum([const20[i]*x[i] for i in var]) == 0

In [23]:
model.solve(solver=GUROBI())

Optimize a model with 20 rows, 18 columns and 51 nonzeros
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [1e-01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 5e+03]
Presolve removed 13 rows and 8 columns
Presolve time: 0.01s
Presolved: 7 rows, 10 columns, 23 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.1097500e+05   1.625000e+03   0.000000e+00      0s
       5    4.0132500e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  4.013250000e+05
Gurobi status= 2


1

In [24]:
print('Status:', model.status)
for i in model.variables():
    print(i.name, '=', i.varValue)
print('Optimal value=', value(model.objective))

Status: 1
X_D1 = 0.0
X_D2 = 4000.0
X_D3 = 0.0
X_I1 = 1500.0
X_I2 = 0.0
X_I3 = 3500.0
X_X11 = 2500.0
X_X12 = 2500.0
X_X13 = 3500.0
X_X21 = 4000.0
X_X22 = 0.0
X_X23 = 2500.0
X_Y11 = 0.0
X_Y12 = 0.0
X_Y13 = 500.0
X_Y21 = 2000.0
X_Y22 = 0.0
X_Y23 = 500.0
Optimal value= 401325.0


In [29]:
print('Sensitivity Analysis')
for name, c in model.constraints.items():
    print('Constraint: {} \tShadow Price={:.2f} \tSlack={}'.format(name, c.pi, c.slack))

Sensitivity Analysis
Constraint: _C1 	Shadow Price=-0.00 	Slack=-0.0
Constraint: _C2 	Shadow Price=-0.00 	Slack=-0.0
Constraint: _C3 	Shadow Price=-0.00 	Slack=20.0


In [26]:
import numpy as np
from pulp import *

model = LpProblem('Giapetto', LpMaximize)

soldiers = LpVariable('soldiers', lowBound=0, cat='Integer')
trains = LpVariable('trains', lowBound=0, cat='Integer')

raw_material_costs = 10*soldiers+9*trains
variable_costs = 14*soldiers+10*trains

revenues = 27*soldiers+21*trains

profit = revenues - (raw_material_costs+variable_costs)
model += profit

carpentry_hours = soldiers+trains
model += (carpentry_hours <= 80)

finishing_hours = 2*soldiers+trains
model += (finishing_hours <= 100)

model += (soldiers <= 40)

print(model)

Giapetto:
MAXIMIZE
3*soldiers + 2*trains + 0
SUBJECT TO
_C1: soldiers + trains <= 80

_C2: 2 soldiers + trains <= 100

_C3: soldiers <= 40

VARIABLES
0 <= soldiers Integer
0 <= trains Integer



In [27]:
# solve the LP using the default solver
optimization_result = model.solve()

# make sure we got an optimal solution
assert optimization_result == LpStatusOptimal

# display the results
for var in (soldiers, trains):
    print('Optimal weekly number of {} to produce: {:1.0f}'.format(var.name, var.value()))

Optimal weekly number of soldiers to produce: 20
Optimal weekly number of trains to produce: 60


In [28]:
from pulp import *

identifiers = ['A', 'B']
obj_coef = dict(zip(identifiers, [3, 2]))
const1 = dict(zip(identifiers, [10, 5]))
const2 = dict(zip(identifiers, [4, 4]))
const3 = dict(zip(identifiers, [2, 6]))
const4 = dict(zip(identifiers, [1, 0]))
const5 = dict(zip(identifiers, [1, -2]))

prob = LpProblem(name='Minimize', sense=LpMaximize)

x = LpVariable.dicts(name='Decision variables', indexs=identifiers, 
                     lowBound=0)

prob += lpSum(x[i] * obj_coef[i] for i in identifiers)

prob += lpSum(x[i] * const1[i] for i in identifiers) <= 300
prob += lpSum(x[i] * const2[i] for i in identifiers) <= 160
prob += lpSum(x[i] * const3[i] for i in identifiers) <= 180
prob += lpSum(x[i] * const4[i] for i in identifiers) >= 20
prob += lpSum(x[i] * const5[i] for i in identifiers) == 0

prob.solve(solver=GUROBI())

print('Status:', prob.status)
for i in prob.variables():
    print(i.name, '=', i.varValue)
print('Optimal value:', value(prob.objective))

Optimize a model with 5 rows, 2 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  9.600000000e+01
Gurobi status= 2
Status: 1
Decision_variables_A = 24.0
Decision_variables_B = 12.0
Optimal value: 96.0
